<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/streamlit101v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# streamlit_app.py

import time
import pandas as pd
import streamlit as st
from io import StringIO
import msal
import json
import requests

# Azure AD Configuration
CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'
TENANT_ID = 'YOUR_TENANT_ID'
AUTHORITY = f'https://login.microsoftonline.com/{TENANT_ID}'
SCOPE = ['User.Read']
REDIRECT_URI = 'http://localhost:8501'  # Update this based on your app's configuration

# Initialize MSAL
def get_msal_app():
    return msal.ConfidentialClientApplication(
        CLIENT_ID,
        authority=AUTHORITY,
        client_credential=CLIENT_SECRET
    )

# Get access token
def get_access_token():
    msal_app = get_msal_app()
    result = msal_app.acquire_token_for_client(scopes=SCOPE)
    return result['access_token'] if 'access_token' in result else None

# Function to fetch user info
def get_user_info(access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get('https://graph.microsoft.com/v1.0/me', headers=headers)
    return response.json() if response.status_code == 200 else None

# Sample data for demonstration
data = pd.DataFrame({
    "options": ["apple", "banana", "grape", "orange", "melon", "watermelon", "kiwi", "pineapple", "strawberry", "blueberry"]
})

# Sample processed file names
processed_files = [f"file_{i}.txt" for i in range(1, 21)]  # 20 sample file names

# Set up the title of the app with the new application name
st.title("InsightFlow")

# CSS for setting background image
st.markdown(
    """
    <style>
    .output-background {
        background-image: url('https://path_to_your_logo/logo.png'); /* Replace with the path to your logo */
        background-size: 200px;
        background-position: center;
        background-repeat: no-repeat;
        opacity: 0.1;
    }
    .download-button-container {
        display: flex;
        justify-content: flex-end; /* Aligns the button to the right */
        margin-bottom: 10px; /* Adds space below the button */
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Authenticate user
if 'access_token' not in st.session_state:
    st.session_state['access_token'] = get_access_token()

# Verify token and get user info
if st.session_state['access_token']:
    user_info = get_user_info(st.session_state['access_token'])
    if user_info:
        st.sidebar.header(f"Welcome, {user_info['displayName']}!")  # Display user name in sidebar
    else:
        st.error("Could not retrieve user information.")
else:
    st.error("Authentication failed.")

# Sidebar layout with menu options
st.sidebar.header("Menu")
menu_options = ["Home", "Settings"]
selected_option = st.sidebar.selectbox("Select an option:", menu_options)

# Content for the Settings page
def display_settings():
    st.header("Settings")

    # Create tabs for Terms and Conditions and About Information
    tab1, tab2 = st.tabs(["Terms and Conditions", "About Information"])

    # Terms and Conditions tab
    with tab1:
        st.write("""
        **Terms and Conditions**: Please read these terms and conditions carefully before using the InsightFlow application.

        1. **Acceptance of Terms**: By using this application, you agree to abide by these terms and conditions.
        2. **Use of Application**: You may use this application only for lawful purposes.
        3. **Limitation of Liability**: InsightFlow is not responsible for any losses or damages incurred while using this application.
        4. **Changes to Terms**: We reserve the right to change these terms at any time without prior notice. Please check this page regularly for updates.
        5. **Contact Us**: If you have any questions about these terms, please contact us at support@insightflow.com.
        """)

    # About Information tab
    with tab2:
        st.write("""
        **About InsightFlow**

        InsightFlow is a powerful application designed to help users manage and analyze their data efficiently.

        - **Version**: 1.0
        - **Developed by**: Your Company Name
        - **Release Date**: October 2024

        InsightFlow provides an intuitive interface, making it easy to navigate through features and functions.

        For any inquiries or support, please contact us at support@insightflow.com.
        """)

# Main application logic based on selected option
if selected_option == "Home":
    # Sidebar layout with an input box, submit button, and an expander
    st.sidebar.header("**Input Section**")  # Making the header bold

    # Incremental search: Filter options based on user input
    user_input = st.sidebar.text_input("Enter something:")
    filtered_options = data[data["options"].str.contains(user_input, case=False, na=False)]["options"].tolist()

    # Show dropdown with filtered options
    selected_input = st.sidebar.selectbox("Select from filtered options:", filtered_options)

    # Expander to show additional details
    with st.sidebar.expander("Show Details"):
        st.write("""
        InsightFlow takes input from the user and displays outputs in three different tabs.
        Please enter your text and click Submit to see the results.
        """)

    # Submit button
    if st.sidebar.button("Submit"):
        # Simulate a time delay with a progress bar
        with st.spinner("Processing..."):
            progress_bar = st.progress(0)
            for i in range(100):
                time.sleep(0.03)  # Simulating a processing time
                progress_bar.progress(i + 1)
            progress_bar.empty()

        # Generate output data for each tab
        output1 = f"Output 1 for input: {selected_input}"
        output2 = f"Output 2 for input: {selected_input}"
        processed_files_content = "\n".join([f"{file}: Details about {file}" for file in processed_files])

        # Right side layout for tabbed display
        tab1, tab2, tab3 = st.tabs(["Output 1", "Output 2", "Processed Files"])

        # Apply background class to main output area
        with st.container():
            st.markdown('<div class="output-background">', unsafe_allow_html=True)

            # Tab 1: Display output and download button
            with tab1:
                st.write(output1)
                st.markdown('<div class="download-button-container">', unsafe_allow_html=True)
                st.download_button(label="Download Output 1", data=StringIO(output1).getvalue(), file_name="output1.txt", mime="text/plain")
                st.markdown('</div>', unsafe_allow_html=True)

            # Tab 2: Display output and download button
            with tab2:
                st.write(output2)
                st.markdown('<div class="download-button-container">', unsafe_allow_html=True)
                st.download_button(label="Download Output 2", data=StringIO(output2).getvalue(), file_name="output2.txt", mime="text/plain")
                st.markdown('</div>', unsafe_allow_html=True)

            # Tab 3: Display processed files and download button
            with tab3:
                st.write(f"Total files processed: {len(processed_files)}")
                for file in processed_files:
                    with st.expander(file):  # Use an expander for each file to support long text
                        st.write(f"Details about {file}: Lorem ipsum dolor sit amet, consectetur adipiscing elit...")  # Placeholder text

                st.markdown('<div class="download-button-container">', unsafe_allow_html=True)
                st.download_button(label="Download Processed Files", data=StringIO(processed_files_content).getvalue(), file_name="processed_files.txt", mime="text/plain")
                st.markdown('</div>', unsafe_allow_html=True)

            st.markdown('</div>', unsafe_allow_html=True)

else:
    display_settings()
